<img src ='./02.png'>

## **1. import Data**

In [4]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import mediapipe as mp
import warnings
warnings.filterwarnings('ignore')
import torch
from facenet_pytorch import MTCNN

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_draw = mp.solutions.drawing_utils

**Function calculate**

In [7]:
def calculateangle(a,b,c):
    a = np.array(a) # First 
    b = np.array(b) # Mid 
    c = np.array(c) # End 

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    triangle = np.abs(radians*180.0/np.pi)

    if triangle > 180.0:
        triangle = 360-triangle
    
    return triangle

**Test Detection and Drawing landmarks**

In [9]:
# Load the pre-trained MTCNN model
mtcnn = MTCNN(keep_all=True, device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

cap = cv2.VideoCapture('./self_train.mp4')
# cap = cv2.VideoCapture(0)

# Setup Mediapipe instance
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) as hands:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Stream stopped.")
                break

            # Perform face detection
            boxes, probs = mtcnn.detect(frame)

            if boxes is not None:
                for box in boxes:
                    x, y, width, height = map(int, box)
                    
                    # size detection face
                    padding_percentage = 0.1  # 10% of the box size
                    padding_x = int((width - x) * padding_percentage)
                    padding_y = int((height - y) * padding_percentage)

                    x -= padding_x
                    y -= padding_y
                    width += 2 * padding_x
                    height += 2 * padding_y

                    cv2.rectangle(frame, (x, y), (width, height), (0, 0, 255), 2)
            
            # Recolor image to RGB
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            results = pose.process(frame)
            results_h = hands.process(frame)

            # Recolor back to BRG
            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                lanmarks = results.pose_landmarks.landmark

                # Get coordinates
                shoulder = [lanmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [lanmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [lanmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                shoulder1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                # Calculate angle
                triangle = calculateangle(shoulder, elbow, wrist).round(2)
                triangle2 = calculateangle(shoulder1, elbow1, wrist1).round(2)
                # print(angle)

                # Visualize triangle shoulder, elbow, wrist
                cv2.putText(frame, str(triangle),
                            (int(elbow[0]*frame.shape[1]), int(elbow[1]*frame.shape[0])),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                
                # Visualize triangle hip, shoulder1, elbow1
                cv2.putText(frame, str(triangle2),
                            (int(elbow1[0]*frame.shape[1]), int(elbow1[1]*frame.shape[0])),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                
                # Curl Counter Logic
                if triangle2 > 150:
                    stage = "down"
                if triangle2 < 30 and stage =='down':
                    stage="up"
                    counter +=1
                    # print(counter)

                # Render curl counter
                # Setup status box
                cv2.rectangle(frame, (0,0), (250,73), (245,117,16), -1)
                
                # Rep data
                cv2.putText(frame, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame, str(counter), 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                
                # Stage data
                cv2.putText(frame, 'STAGE', (65,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame, stage, 
                            (60,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

                # Render detections
                mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
                    # landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                
                if results_h.multi_hand_landmarks:
                    for hand_landmarks in results_h.multi_hand_landmarks:
                        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            except:
                pass

            cv2.imshow('MediaPipe Pose', frame)
            
            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

## **2. Capture landmarks & Create DataFrame**

In [10]:
landmarks_2 = ['class']
for val in range(1,33+1):
    landmarks_2 += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [11]:
import csv

In [12]:
with open('coords_film.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks_2)

In [13]:
def export_landmarks(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)

        with open('coords_film.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

**Record Dataframe From Pose Landmarks**
- Record manual

In [9]:
cap = cv2.VideoCapture('./thank.mp4')
cap = cv2.VideoCapture('./self_train.mp4')
# cap = cv2.VideoCapture(0)

# Setup Mediapipe instance
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) as hands:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Stream stopped.")
                break

            # Recolor image to RGB
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            results = pose.process(frame)
            results_h = hands.process(frame)

            # Recolor back to BRG
            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                lanmarks = results.pose_landmarks.landmark

                # Get coordinates
                shoulder = [lanmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [lanmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [lanmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                shoulder1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                # Calculate angle
                triangle = calculateangle(shoulder, elbow, wrist).round(2)
                triangle2 = calculateangle(shoulder1, elbow1, wrist1).round(2)
                # print(angle)

                # Visualize triangle shoulder, elbow, wrist
                cv2.putText(frame, str(triangle),
                            (int(elbow[0]*frame.shape[1]), int(elbow[1]*frame.shape[0])),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                
                # Visualize triangle shoulder1, elbow1, wrist1
                cv2.putText(frame, str(triangle2),
                            (int(elbow1[0]*frame.shape[1]), int(elbow1[1]*frame.shape[0])),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                
                # Curl Counter Logic
                if triangle2 > 150:
                    stage = "down"
                if triangle2 < 30 and stage =='down':
                    stage="up"
                    counter +=1
                    # print(counter)
                
                # Render curl counter
                # Setup status box
                cv2.rectangle(frame, (0,0), (250,73), (245,117,16), -1)
                
                # Rep data
                cv2.putText(frame, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame, str(counter), 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                
                # Stage data
                cv2.putText(frame, 'STAGE', (65,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame, stage, 
                            (80,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

                # Render detections
                mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
                    # landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                
                if results_h.multi_hand_landmarks:
                    for hand_landmarks in results_h.multi_hand_landmarks:
                        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            except:
                pass
                

            cv2.imshow('MediaPipe Pose', frame)
            
            k = cv2.waitKey(1)
            if k == ord('1'):
                export_landmarks(results, 'want')
            if k == ord('2'):
                export_landmarks(results, "don't want")
            if k == ord('3'):
                export_landmarks(results, 'more')
            if k == ord('4'):
                export_landmarks(results, 'potty')
            if k == ord('5'):
                export_landmarks(results, 'dirty')
            if k == ord('6'):
                export_landmarks(results, 'clean')
            if k == ord('7'):
                export_landmarks(results, 'home')
            if k == ord('8'):
                export_landmarks(results, 'school') 
            if k == ord('9'):
                export_landmarks(results, 'play')
            if k == ord('0'):
                export_landmarks(results, 'bed')  

            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

Stream stopped.


## **3. Train Machine Learning Classification Model**

**3.1 Read in Collected Data and Process**

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [20]:
df_film = pd.read_csv('coords_film.csv')

In [21]:
df_new = pd.read_csv('coords_new.csv')

In [22]:
df_thank = pd.read_csv('coords_thank.csv')

In [23]:
df_ori = pd.read_csv('coords_ori.csv')

In [24]:
df_train = pd.concat([df_film, df_new, df_thank,df_ori], ignore_index=True)

In [25]:
df_train

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,want,0.523251,0.450990,-0.459594,0.999819,0.533429,0.417930,-0.431221,0.999746,0.539272,...,0.297255,0.010767,0.526460,1.816151,-0.012140,0.006274,0.441641,1.810452,0.018783,0.009043
1,want,0.524013,0.449159,-0.515879,0.999908,0.534502,0.416850,-0.491419,0.999874,0.540472,...,0.423895,0.005330,0.554579,1.776513,0.108589,0.003069,0.477024,1.773829,0.141674,0.004596
2,want,0.524378,0.448990,-0.524086,0.999916,0.534832,0.416744,-0.500241,0.999885,0.540848,...,0.435331,0.004874,0.554528,1.776742,0.098729,0.002805,0.476732,1.773951,0.149873,0.004227
3,want,0.525125,0.448920,-0.509409,0.999923,0.535406,0.416669,-0.486281,0.999894,0.541555,...,0.419291,0.004455,0.553707,1.776260,0.100837,0.002565,0.476286,1.773094,0.135557,0.003885
4,want,0.525927,0.448911,-0.530385,0.999929,0.535953,0.416656,-0.506059,0.999903,0.542202,...,0.417702,0.004097,0.551577,1.776931,0.101532,0.002363,0.473619,1.773873,0.128202,0.003605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12569,bed,0.582735,0.231646,-0.549904,0.999665,0.593195,0.194960,-0.514371,0.999619,0.601733,...,0.483065,0.003722,0.601851,1.825967,0.144897,0.002096,0.512101,1.814257,0.183510,0.004054
12570,bed,0.583797,0.228436,-0.610396,0.999688,0.594404,0.193720,-0.574524,0.999641,0.602766,...,0.510541,0.004981,0.603092,1.825737,0.128428,0.002894,0.512202,1.814188,0.213240,0.005547
12571,bed,0.585292,0.225537,-0.606941,0.999693,0.595918,0.191964,-0.570239,0.999643,0.604145,...,0.498834,0.005538,0.607218,1.827818,0.120828,0.003624,0.515186,1.817711,0.201774,0.006492
12572,bed,0.585755,0.223811,-0.567971,0.999695,0.596428,0.191312,-0.531835,0.999640,0.604642,...,0.408466,0.005802,0.612995,1.827712,0.118651,0.004350,0.526171,1.818134,0.122340,0.006794


In [26]:
df_train['class'] = df_train['class'].replace("don't want", "don't_want")

In [27]:
df_train['class'].unique()

array(['want', "don't_want", 'more', 'potty', 'dirty', 'clean', 'home',
       'school', 'play', 'bed'], dtype=object)

In [28]:
df_train['class'].value_counts()

class
don't_want    1576
want          1512
potty         1489
dirty         1339
more          1309
home          1137
clean         1114
bed           1097
play          1023
school         978
Name: count, dtype: int64

In [29]:
X = df_train.drop('class', axis=1)
y = df_train['class']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [31]:
y_test

1164          potty
1298          potty
9549     don't_want
2447         school
10721    don't_want
            ...    
7249          potty
8086          potty
8744           want
9740           more
12393          play
Name: class, Length: 2515, dtype: object

**3.2 Train Machine Learning Classification Model**

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [33]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
    'svm':make_pipeline(StandardScaler(), SVC())
}

In [34]:
train_models = {}
for ml, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    train_models[ml] = model

In [35]:
train_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())]),
 'svm': Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])}

In [36]:
train_models['lr'].predict(X_test)

array(['potty', 'potty', "don't_want", ..., 'want', 'more', 'play'],
      dtype=object)

### **3.3 Evaluate & Select Model**

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [38]:
for ml, model in train_models.items():
    y_pred = model.predict(X_test)
    print(ml, accuracy_score(y_test.values, y_pred), 
          precision_score(y_test.values, y_pred, average='weighted'), # average='micro','macro', 'weighted' / pos_label='class name'
          recall_score(y_test.values, y_pred, average='weighted')) # average='micro','macro', 'weighted' / pos_label='class name'

lr 0.9852882703777336 0.9853582795373249 0.9852882703777336
rc 0.9614314115308151 0.962682110645092 0.9614314115308151
rf 0.9992047713717693 0.9992098853822403 0.9992047713717693
gb 0.9944333996023856 0.9944553458651288 0.9944333996023856
svm 0.9892644135188867 0.9893318684020387 0.9892644135188867


In [39]:
y_pred = train_models['lr'].predict(X_test)

In [40]:
y_pred[:10]

array(['potty', 'potty', "don't_want", 'school', "don't_want",
       "don't_want", 'potty', 'clean', 'home', 'dirty'], dtype=object)

In [41]:
y_test[:10]

1164          potty
1298          potty
9549     don't_want
2447         school
10721    don't_want
10707    don't_want
11907         potty
3880          clean
5733           home
4524          dirty
Name: class, dtype: object

In [42]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[209   0   0   0   1   0   0   0   0   1]
 [  0 241   0   0   0   0   0   0   2   0]
 [  0   0 250   0   0   0   0   0   0   0]
 [  2   0   0 303   0   1   1   0   0   2]
 [  2   0   0   0 225   0   0   5   2   0]
 [  0   0   0   5   0 264   0   0   0   2]
 [  0   0   0   1   0   0 192   1   0   0]
 [  0   0   1   0   0   0   0 291   0   0]
 [  0   1   0   0   0   0   0   0 206   0]
 [  0   0   0   2   0   4   0   1   0 297]]


In [43]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         bed       0.98      0.99      0.99       211
       clean       1.00      0.99      0.99       243
       dirty       1.00      1.00      1.00       250
  don't_want       0.97      0.98      0.98       309
        home       1.00      0.96      0.98       234
        more       0.98      0.97      0.98       271
        play       0.99      0.99      0.99       194
       potty       0.98      1.00      0.99       292
      school       0.98      1.00      0.99       207
        want       0.98      0.98      0.98       304

    accuracy                           0.99      2515
   macro avg       0.99      0.99      0.99      2515
weighted avg       0.99      0.99      0.99      2515



---

In [40]:
# with open('model2_3.pkl', 'wb') as f:
#     pickle.dump(svc, f)

In [44]:
with open('model_pose.pkl', 'wb') as f:
    pickle.dump(train_models['lr'], f)

## **4. Make Detections with Model**

In [45]:
import pickle

In [46]:
with open('model_pose.pkl', 'rb') as f:
    model = pickle.load(f)

In [47]:
# Load the pre-trained MTCNN model
mtcnn = MTCNN(keep_all=True, device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'))

cap = cv2.VideoCapture('./datatest.mp4')
# cap = cv2.VideoCapture(0)

# # Get the original frame rate
original_frame_rate = cap.get(cv2.CAP_PROP_FPS)

# # # Set the frame rate of the capture to the original frame rate
cap.set(cv2.CAP_PROP_FPS, original_frame_rate)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter('Detection2_2.mp4', fourcc, original_frame_rate, (int(cap.get(3)), int(cap.get(4))))

# Setup Mediapipe instance
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:
    with mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) as hands:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Stream stopped.")
                break

            # Perform face detection
            boxes, probs = mtcnn.detect(frame)

            if boxes is not None:
                for box in boxes:
                    x, y, width, height = map(int, box)
                    
                    padding_percentage = 0.1  # 10% of the box size
                    padding_x = int((width - x) * padding_percentage)
                    padding_y = int((height - y) * padding_percentage)

                    x -= padding_x
                    y -= padding_y
                    width += 2 * padding_x
                    height += 2 * padding_y

                    cv2.rectangle(frame, (x, y), (width, height), (0, 0, 255), 2)
            
            # Recolor image to RGB
            frame.flags.writeable = False
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            results = pose.process(frame)
            results_h = hands.process(frame)

            # Recolor back to BRG
            frame.flags.writeable = True
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                lanmarks = results.pose_landmarks.landmark

                # Get coordinates
                shoulder = [lanmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [lanmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [lanmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, lanmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                shoulder1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                elbow1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                wrist1 = [lanmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, lanmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                # Calculate angle
                triangle = calculateangle(shoulder, elbow, wrist).round(2)
                triangle2 = calculateangle(shoulder1, elbow1, wrist1).round(2)
                # print(angle)

                # Visualize triangle shoulder, elbow, wrist
                cv2.putText(frame, str(triangle),
                            (int(elbow[0]*frame.shape[1]), int(elbow[1]*frame.shape[0])),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                
                # Visualize triangle shoulder1, elbow1, wrist1
                cv2.putText(frame, str(triangle2),
                            (int(elbow1[0]*frame.shape[1]), int(elbow1[1]*frame.shape[0])),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
                
                # Curl Counter Logic
                if triangle2 > 155:
                    stage = "down"
                if triangle2 < 30 and stage =='down':
                    stage="up"
                    counter +=1
                    
                if stage == "up":
                    row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                    X = pd.DataFrame([row], columns=landmarks_2[1:])
                    body_class = model.predict(X)[0]
                    prob = model.predict_proba(X)[0]
                    # print(body_class, prob)
                if stage == "down":
                    body_class = 'Stand'
                    prob = 1.0
                    

                # Render detections
                mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
                    # landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
                
                if results_h.multi_hand_landmarks:
                    for hand_landmarks in results_h.multi_hand_landmarks:
                        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # Render curl counter
                # Setup status box
                cv2.rectangle(frame, (0,0), (250,80), (245,117,16), -1)
                
                # Rep data
                cv2.putText(frame, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame, str(counter), 
                            (10,65), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                
                # Stage data
                cv2.putText(frame, 'STAGE', (65,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame, stage, 
                            (90,65), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
                
                #  Display class
                cv2.rectangle(frame, (600,0), (250,80), (0,0,255), -1)
                cv2.putText(frame, 'CLASS',
                            (300,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame, body_class.split(' ')[0],
                            (285,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                
                # Display Probbability
                cv2.putText(frame, 'PROB',
                            (500,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(frame, str(round(prob[np.argmax(prob)], 2)),
                            (500,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
                
                # Display on Face Detection
                cv2.putText(frame, f'Class: {body_class.split(" ")[0]}', (x, y - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, f'Prob: {str(round(prob[np.argmax(prob)], 2))}', (x, y+260), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

            except:
                pass

            # Write the frame to the output video file
            output_video.write(frame)

            cv2.imshow('MediaPipe Pose', frame)
            
            if cv2.waitKey(1) == ord('q'):
                break

        cap.release()
        output_video.release()
        cv2.destroyAllWindows() 

### **Group 8**
1. นายธนภัทร ศรีเพ็ชร 66076020
2. กนกศักดิ์ เชาว์เลิศ 66076002
3. ปวินกาน สารททอง 66076028

---
---

In [48]:
cap = cv2.VideoCapture('./Detection2_2.mp4')
# Setup Mediapipe instance
    
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Stream stopped.")
        break
    
    cv2.imshow('MediaPipe Pose', frame)
    
    if cv2.waitKey(20) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()